In [2]:
from bs4 import BeautifulSoup
import sys
import time as tm
from datetime import datetime, time
import os
import requests
import codecs
import json
from optparse import OptionParser
key = 'Z4Y7a0W330WT1m6DUoHXhQ'
#url = 'http://realtime.mbta.com/developer/api/v2/predictionsbystop?api_key=wX9NwuHnZU2ToO7GmGR9uw&stop=place-bbsta&format=json'

* "70154","70154","Copley - Inbound","",42.349974,-71.077447,"","",0,"place-coecl",0

* "70155","70155","Copley - Outbound","",42.349974,-71.077447,"","",0,"place-coecl",0

1. * Route_type = 0 -> subway
   * 1 -> rail way
   * 2 -> bus
   
2. * 39 bus "175","175","Boylston St @ Dartmouth St","",42.350011,-71.077432,"","",0,"",0



In [3]:
def get_stop_info(stop_id):
    stop = str(stop_id)
    url = 'http://realtime.mbta.com/developer/api/v2/predictionsbystop?'\
          'api_key='+key+'&stop='+stop+'&format=json'
    response = requests.get(url) 
    #text = response.text.encode('utf-8')
    data = json.loads(response.text)
    return data

In [7]:
#data.keys() -> ['mode', 'stop_id', 'stop_name', 'alert_headers']

In [4]:
def get_bus_info(stop_data,t_type,route_n):
    if 'mode' not in stop_data.keys():
        print('service goes offline @ %d:%d'%(datetime.now().hour,datetime.now().minute))
    else:
        for i in stop_data['mode']:
            if i['mode_name'] == t_type:
                for route in i['route']:
                    if route['route_name']==str(route_n):
                        return route['direction'][0]['trip']
    return 'This bus is unavailable or bus id not found.'

In [10]:
#bus_data.keys() -> ['route_name', 'direction', 'route_id']

In [11]:
#bus_data['direction'][0].keys() -> ['trip', 'direction_name', 'direction_id']

In [12]:
# parser = OptionParser()
# parser.add_option('-v', '--verbose', action='store_true')
# parser.add_option('-o', '--output-file')
# parser.add_option('-n', '--n_of_time', type = 'int')
# parser.add_option('-s', '--desired_stop', type='str')
# parser.add_option('-t', '--t_type', type='str')
# parser.add_option('-r', '--route_n', type='int')
# option, args = parser.parse_args()
# if not option.output_file:
#     print('specify input and output files')
#     parser.print_help()
#     sys.exit(1)
# if not (option.desired_stop and option.t_type and option.route_n):
#     print('specify correct input')
#     parser.print_help()
#     sys.exit(1)


In [13]:
# if __name__ == "__main__":
#     desired_stop = sys.argv[1]
#     t_type = sys.argv[2]

In [7]:
#import pickle
def writer(data,filename):
    with open(filename+'.txt', "w") as fp: 
        json.dump(data, fp)
    #print(filename+' is written @ '+str(datetime.now()))
    return str(filename+' is written @ '+str(datetime.now()))

In [ ]:
# def worker(n,desired_stop,t_type,route_n,filename):
#     count = 0
#     result = []
#     progress = ProgressBar(n, fmt=ProgressBar.FULL)
#     while(count<n):
#         progress.current += 1
#         progress()
#         stop_data = get_stop_info(desired_stop)
#         bus_list = get_bus_info(stop_data,t_type,route_n)
#         #print([count,bus_list])
#         result.append([count,datetime.now(),bus_list])
#         count+=1
#         tm.sleep(10)
#     progress.done()
#     writer(result,filename)
#     return result
        
# #worker(1800,171,'Bus',39,'test.txt')

In [10]:
from progressbar import*
now = datetime.now()
now_time = now.time()
n,desired_stop, t_type, route_n = 30,175,'Bus',39
#    input("Please enter parameters in format of: n, stop_id, type, route_n =>").split(',')
count = 0
while(True):
    if (time(4,30) <= now.time() <= time(23,59)) or (time(0,0) <= now.time() <= time(1,0)):
        filename = 'output/'+tm.strftime("%b")+str(datetime.now().day)+'_'
        saved_ = filename+"%d" %(count//int(n))
        curr = 0
        result = {}
        progress = ProgressBar(int(n), fmt=ProgressBar.FULL)
        while(curr<int(n)):
            progress.current += 1
            progress()
            stop_data = get_stop_info(desired_stop)
            bus_list = get_bus_info(stop_data,t_type,route_n)
            #print([count,bus_list])
            result[count]={"time":tm.time(),"info":bus_list}
            curr+=1
            count+=1
            tm.sleep(10)
        progress.done()
        print('Data_%d collected, writting to file: ' %(count//int(n)-1)+saved_ )
        msg = writer(result,saved_)
        if msg:
            print(msg)

[========================================] 30/30 (100%)  0 to go
[=                                       ]  1/30 (  3%) 29 to go

Data_0 collected, writting to file: output/Jul5_0
output/Jul5_0 is written @ 2017-07-05 18:51:49.819585


[========================================] 30/30 (100%)  0 to go
[=                                       ]  1/30 (  3%) 29 to go

Data_1 collected, writting to file: output/Jul5_1
output/Jul5_1 is written @ 2017-07-05 18:56:55.910129


[========================================] 30/30 (100%)  0 to go
[=                                       ]  1/30 (  3%) 29 to go

Data_2 collected, writting to file: output/Jul5_2
output/Jul5_2 is written @ 2017-07-05 19:02:00.206124


[========================================] 30/30 (100%)  0 to go
[=                                       ]  1/30 (  3%) 29 to go

Data_3 collected, writting to file: output/Jul5_3
output/Jul5_3 is written @ 2017-07-05 19:07:12.420136


[=========                               ]  7/30 ( 23%) 23 to go

ConnectionError: HTTPConnectionPool(host='realtime.mbta.com', port=80): Max retries exceeded with url: /developer/api/v2/predictionsbystop?api_key=Z4Y7a0W330WT1m6DUoHXhQ&stop=175&format=json (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x10708c588>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [ ]:
# #by default 
# desired_stop = 175
# t_type = 'Bus'
# route_n = 39